# WSI - Ćwiczenie 2

*Autor - Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest zaimplementowanie algorytmu ewolucyjnego:
- z selekcją turniejową ($k=2$)
- krzyżowaniem jednopunktowym
- mutacją gaussowską
- sukcesją generacyjną

Należy również wykorzystać ten algorytm do znalezienia minimów dwóch funkcji:
$$f_1(x_1,y_1)=(x_1^2+y_1-11)^2+(x_1+y_1^2-7)^2$$
$$f_2(x_2,y_2)=2*x_2^2+1.05*x_2^4+\frac{x_2^6}{6}+x_2y_2+y_2^2$$
dla dziedziny $D=[-5,5]\times[-5,5]$, oraz zbadać wpływ dystrybucji populacji początkowej, prawdopodobieństwa mutacji oraz prawdopodobieństwo krzyżowania na współrzędne znalezionych minimów funkcji $f_1$.